In [47]:
import chemsource as cs
import pandas as pd
import numpy as np
from tqdm import tqdm
from ast import literal_eval
from helpers.classify import *
from helpers.classify import *
from helpers.visualization import *


from pandarallel import pandarallel

from helpers.preprocessing import *
pandarallel.initialize(progress_bar=True, nb_workers=15)
tqdm.pandas()

INFO: Pandarallel will run on 15 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [37]:
food_data = pd.read_csv("../data/cmmc/curated_food_molecules.tsv", sep="\t")

In [38]:
BASE_PROMPT = "You are a helpful scientist that will classify the provided compounds using only the information provided as any combination of the following: MEDICAL, ENDOGENOUS, FOOD, PERSONAL CARE, INDUSTRIAL. Note that MEDICAL refers to compounds actively used as approved medications in humans or in late-stage clinical trials in humans. Note that ENDOGENOUS refers to compounds that are produced by the human body specifically. ENDOGENOUS excludes essential nutrients that cannot be synthesized by the human body. Note that FOOD refers to compounds present in natural food items or food additives. Note that INDUSTRIAL should be used only for synthetic compounds not used as a contributing ingredient in the medical, personal care, or food industries. Note that PERSONAL CARE refers to non-medicated compounds typically used for activities such as skincare, beauty, and fitness. Specify INFO instead if more information is needed. DO NOT MAKE ANY ASSUMPTIONS, USE ONLY THE INFORMATION PROVIDED AFTER THE COMPOUND NAME BY THE USER. A classification of INFO will also be rewarded when correctly applied and is strongly encouraged if information is of poor quality, if there is not enough information, or if you are not completely confident in your answer.  Provide the output as a plain text separated by commas, and provide only the categories listed (either list a combination of INDUSTRIAL, ENDOGENOUS, PERSONAL CARE, MEDICAL, FOOD or list INFO), with no justification. Provided Information:\n"
deepinfra_key = pd.read_csv("../secrets/deepinfra_api.txt", header=None, index_col=None).values[0][0]
model_deepseek_v3 = cs.ChemSource()
model_deepseek_v3.configure(
    model="deepseek-ai/DeepSeek-V3",
    model_api_key=deepinfra_key,
    prompt=BASE_PROMPT,
    temperature=0,
    top_p=1,
    logprobs=True,
    base_url="https://api.deepinfra.com/v1/openai")


In [39]:
food_data["input_name"] = food_data["input_name"].apply(lambda x : [x])
food_data["text"] = food_data["input_name"].parallel_apply(lambda x: list_retrieve(x, model_deepseek_v3))
food_data["cleaned_text"] = food_data["text"].apply(lambda x: x[1][1] if x is not None else None)

food_data["text"] = food_data.parallel_apply(lambda x: list_retrieve(preprocess_chemical(x["input_name"]), model_deepseek_v3) if x["cleaned_text"] is None else x["text"], axis=1)
food_data["cleaned_text"] = food_data["text"].apply(lambda x: x[1][1] if x is not None else None)


/home/prajitrr/miniconda3/envs/chemsource/lib/python3.13/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/prajitrr/miniconda3/envs/chemsource/lib/python3.13/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


/home/prajitrr/miniconda3/envs/chemsource/lib/python3.13/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/prajitrr/miniconda3/envs/chemsource/lib/python3.13/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [43]:
food_data["cleaned_name"] = food_data["text"].apply(lambda x: x[0] if x is not None else None)


In [44]:
food_data

,input_usi,input_name,input_structure,input_partial_structure,input_molecule_origin,input_original_molecule_name,input_original_molecule_structure,input_microbe_ncbi,input_microbe_name,input_microbe_DNARNA,...,database_id,input_confirmation,input_confirmation_description,createtime,inchikey,compound_inchikey,npatlas_url,text,cleaned_text,cleaned_name
0,mzspec:GNPS:GNPS-LIBRARY:accession:CCMSLIB0001...,[(R)-Oxypeucedanin],CC1(C)O[C@@H]1COc2c3ccoc3cc4OC(=O)C=Cc24,NaN,Diet,NaN,NaN,NaN,NaN,NaN,...,c0fbfdf61b28442dadd0b3ef159ef01a-8fe27706-392f...,Not applicable,NaN,2025-01-31T00:19:42.784350,QTAGQHZOLRFCBU-CYBMUJFWSA-N,NaN,No NPAtlas,"((r)-oxypeucedanin, (None, None))",None,(r)-oxypeucedanin
1,mzspec:GNPS:GNPS-LIBRARY:accession:CCMSLIB0001...,[1-Caffeoylquinic acid],O[C@@H]1CC(C[C@@H](O)C1O)(OC(=O)\C=C\c2ccc(O)c...,NaN,Diet,NaN,NaN,NaN,NaN,NaN,...,fd57221dda994c3cb62795a54ce7ca6c-ef81126a-d881...,Not applicable,NaN,2025-01-30T22:32:50.133244,GWTUHAXUUFROTF-AVXJPILUSA-N,NaN,No NPAtlas,"(1-caffeoylquinic acid, (None, None))",None,1-caffeoylquinic acid
2,mzspec:GNPS:GNPS-LIBRARY:accession:CCMSLIB0000...,"[1,7-Dimethyluric acid]",CN1C(=O)NC2=C(N(C)C(=O)N2)C1=O,NaN,Diet,NaN,NaN,NaN,NaN,NaN,...,fa5720ff3e6845f7bb352845ac178c2f-ba40e3ae-1a51...,Not applicable,NaN,2025-01-30T23:15:18.514406,NOFNCLGCUJJPKU-UHFFFAOYSA-N,NaN,No NPAtlas,"(1,7-dimethyluric acid, (None, None))",None,"1,7-dimethyluric acid"
3,mzspec:GNPS:GNPS-LIBRARY:accession:CCMSLIB0000...,[2-acetyl-2-thiazoline],CC(=O)C1=NCCS1,NaN,Diet,NaN,NaN,NaN,NaN,NaN,...,c0fbfdf61b28442dadd0b3ef159ef01a-3feabd1c-58e6...,Not applicable,NaN,2025-01-31T00:32:05.426192,FZOZFDAMVVEZSJ-UHFFFAOYSA-N,NaN,No NPAtlas,"(2-acetyl-2-thiazoline, (None, None))",None,2-acetyl-2-thiazoline
4,mzspec:GNPS:GNPS-LIBRARY:accession:CCMSLIB0000...,"[2-amino-1-methyl-6-phenylimidazo [4,5-b] pyri...",Cn1c(N)nc2ncc(cc12)c3ccccc3,NaN,Diet,NaN,NaN,NaN,NaN,NaN,...,fa5720ff3e6845f7bb352845ac178c2f-ef18deda-6b2f...,Not applicable,NaN,2025-01-30T23:16:01.004128,UQVKZNNCIHJZLS-UHFFFAOYSA-N,NaN,No NPAtlas,"(2-amino-1-methyl-6-phenylimidazo [4,5-b] pyri...",None,"2-amino-1-methyl-6-phenylimidazo [4,5-b] pyrid..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,mzspec:GNPS:GNPS-LIBRARY:accession:CCMSLIB0000...,[Vitexin-2-O-rhamnoside],C[C@@H]1O[C@@H](O[C@@H]2[C@@H](O)[C@H](O)[C@@H...,NaN,Diet,NaN,NaN,NaN,NaN,NaN,...,c0fbfdf61b28442dadd0b3ef159ef01a-90417ab1-1d19...,Not applicable,NaN,2025-01-31T00:26:08.820194,LYGPBZVKGHHTIE-HUBYJIGHSA-N,NaN,No NPAtlas,"(Vitexin-2-o-rhamnoside, (None, None))",None,Vitexin-2-o-rhamnoside
291,mzspec:GNPS:GNPS-LIBRARY:accession:CCMSLIB0000...,[Xanthurenic acid],OC(=O)c1cc(O)c2cccc(O)c2n1,NaN,Diet; Host,NaN,NaN,NaN,NaN,NaN,...,fa5720ff3e6845f7bb352845ac178c2f-acdfc786-12cb...,Not applicable,NaN,2025-01-30T23:20:36.777177,FBZONXHGGPHHIY-UHFFFAOYSA-N,NaN,No NPAtlas,"(Xanthurenic acid, (WIKIPEDIA, Xanthurenic aci...","Xanthurenic acid, or xanthurenate, is a metabo...",Xanthurenic acid
292,mzspec:GNPS:GNPS-LIBRARY:accession:CCMSLIB0000...,[xylitol],OC[C@H](O)C(O)[C@H](O)CO,NaN,"Diet; De novo biosynthesis by microbes (e.g., ...",NaN,NaN,NaN,NaN,NaN,...,fd57221dda994c3cb62795a54ce7ca6c-5937c852-ef66...,Not applicable,NaN,2025-01-30T22:13:50.706975,HEBKCHPVOIAQTA-NGQZWQHPSA-N,NaN,No NPAtlas,"(xylitol, (WIKIPEDIA, Xylitol is a chemical co...",Xylitol is a chemical compound with the formul...,xylitol
293,mzspec:GNPS:GNPS-LIBRARY:accession:CCMSLIB0000...,[Xylose],OC[C@@H](O)[C@H](O)[C@@H](O)C=O,NaN,Diet,NaN,NaN,NaN,NaN,NaN,...,fd57221dda994c3cb62795a54ce7ca6c-fbdb6a93-5523...,Not applicable,NaN,2025-01-30T22:13:53.356679,PYMYPHUHKUWMLA-VPENINKCSA-N,NaN,No NPAtlas,"(Xylose, (WIKIPEDIA, Xylose (cf. Ancient Greek...","Xylose (cf. Ancient Greek: ξύλον, xylon, ""wood...",Xylose


In [45]:
food_data["chemsource_output"] = food_data.progress_apply(lambda x: 
                                                          classify_with_log_probs(x["cleaned_name"],x["cleaned_text"], model_deepseek_v3) if x["cleaned_text"] is not None else None, 
                                                          axis=1)

100%|██████████| 295/295 [04:01<00:00,  1.22it/s]


In [49]:
food_data["chemsource_output_classification"] = food_data["chemsource_output"].apply(lambda x: x[0] if x is not None else None)
food_data["chemsource_output_classprobs"]= food_data["chemsource_output"].apply(lambda x: evaluate_probs(x[1]) if x is not None else None)


In [51]:
food_data.to_csv("../data/cmmc/cmmc_food_classified.csv")